<h1 style="color:#4E2A84;">AWS - Braket - Rigetti - Swap Example</h1>

This notebook provides example of using swap gates to swap qubits' state with idle ancilla qubits. As a potential application of this swap operation, this example shows how to sequentially excute multiple copies of a circuit by inserting the swap gates between each copy of the circuit. The swap operations effectivley reset the qubits since the |0> state of the idle ancilla is swapped into the original qubits, allowing new copy of the circuit to execute on the original qubits.

This notebook leverages the `AWS` cloud service, it uses the `Braket` development environment and targets a `Rigetti` superconducting quantum computer.

In [ ]:
# reset all notebook variables just in case
%reset -f

In [ ]:
# general imports
import matplotlib.pyplot as plt

# magic word for producing visualizations in notebook
%matplotlib inline
import string

# AWS imports: import Braket SDK modules
from braket.circuits import Circuit, Gate, Observable
from braket.devices import LocalSimulator
from braket.aws import AwsDevice, AwsQuantumTask
from braket.error_mitigation import Debias

In [ ]:
# sample circuit
circ = Circuit()

circ.h(0)
circ.cnot(0,1)

# print circuit for visual inspection
print(circ)

In [ ]:
# simulate the circuit
device = LocalSimulator()
result = device.run(circ, shots=1000).result()

# show measurement results
counts = result.measurement_counts
print("Measurement counts:", counts)

In [ ]:
# set how many qubits does the input circuit uses and index of first qubit
num_qubits = 2
start_index = 0


circ_with_swaps = Circuit()
circ_with_swaps += circ

curr_index = start_index

num_copies = 2

for i in range(num_copies-1): # subtract one because original circuit is the 1st copy, adding num_copies - 1 after that
    for j in range(num_qubits):
        from_qubit = start_index + j
        to_qubit = num_qubits * (i+1) + j + curr_index
        circ_with_swaps.swap(from_qubit,to_qubit)
        
    circ_with_swaps += circ
    print()
        

print(circ_with_swaps)

In [ ]:
# simulate the circuit
device = LocalSimulator()
result = device.run(circ_with_swaps, shots=1000).result()

# show measurement results
counts = result.measurement_counts
print("Measurement counts:", counts)

In [ ]:
# import json to help pretty print dictionaries
import json

# helper function to check if nth bit is set in a string representing a binary number
# note bit counting starts at 0, i.e. 0th bit is the least significant bit
def is_nth_bit_set(binary_str, n):
    # Reverse the string to access LSB at index 0
    reversed_str = binary_str[::-1]

    if n >= len(reversed_str):
        return False  # Bit doesn't exist

    return reversed_str[n] == '1'

# helper function to check if bits at specific position match a pattern
def is_bit_pattern_set(binary_str, start, end, pattern_str):
    return binary_str[start:end] == pattern_str

counts_separated = dict()

for i in range(num_copies):
    
    temp_counts = dict()
    
    for j in range(2**num_qubits):
        binary_str = bin(j)[2:].zfill(num_qubits)
        temp_counts[binary_str] = 0
    
    for j in range(2**num_qubits):
        for key in counts.keys():
            binary_str = bin(j)[2:].zfill(num_qubits)
            if is_bit_pattern_set(key, i*num_qubits, (i+1)*num_qubits, binary_str):
            # print(i,j,binary_str)
                temp_counts[binary_str] = temp_counts[binary_str] + counts[key]
    
    counts_separated[i] = temp_counts

print(json.dumps(counts_separated, indent=4))

<h1 style="color:#4E2A84;">Estimate qBraids Credits Needed</h1> 

The cost to run a circuit on a quantum computer is divided into per-task cost (effectively the cost to submit a circuit for execution) and per-shot cost (additional cost for each shot).

You can view the cost by selecting $DEVICES$ on the right-hand side of the editor, then search for Rigetti Ankaa-3 (AWS) and finally selecting the pricing cell. <img src="images/image_pricing.png" alt="View pricing" style="width:300px;"/>

The current cost is $30$ qBraid Credits per-task on Ankaa-3 and then $0.09$ qBraid Credits per-task.

Currently each qBraid credit is valued at $0.01$ USD. This means that $100$ credits equate to $1.00$ USD.

In [ ]:
# set number of shots
num_shots = 1000

In [ ]:
# compute cost
perTaksCost = 30.00
perShotCost = 0.09
totalCost = perTaksCost+perShotCost*num_shots

# print cost
print(f'Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:')
print(f'{totalCost} qBraid Credit or equivalently ${totalCost/100} USD')

<h1 style="color:#4E2A84;">Estimate Cost Saving of Running N Copies</h1>

Check the cost to execute the circuit when multiple circuits are run sequentially, and total number of shots is reduced.

In [ ]:
# new number of shots
import math

new_num_shots = math.ceil(num_shots/num_copies)

In [ ]:
# print number of copies being used
print('Number of copies:',num_copies)
print()

print('Original number of shots:',num_shots)
print()

print('Reduced number of shots:',new_num_shots)
print()

# compute cost
perTaksCost = 30.00
perShotCost = 0.09
totalCostWithCopies = perTaksCost+perShotCost*new_num_shots

# print cost
print(f'Cost to run Bell circuit on Rigetti Ankaa-3 via AWS is:')
print(f'{totalCostWithCopies} qBraid Credit or equivalently ${totalCostWithCopies/100} USD')
print()

# savings
print(f'Savings ${totalCost/100 - totalCostWithCopies/100} USD or {100*(totalCost/100 - totalCostWithCopies/100)/(totalCost/100)}%')

<h1 style="color:#4E2A84;">Submit the Circuit to Rigetti Ankaa-3 and Obtain the Results</h1>

You can now submit the circuit for execution.

In [ ]:
# circuit to run
circ_to_run = circ_with_swaps

# print circuit
print(circ_to_run)

In [ ]:
# set up device
ankaa = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Ankaa-3")

# run circuit
ankaa_task = ankaa.run(circ_to_run, shots=1000, disable_qubit_rewiring=False)
    
# get id and status of submitted task
ankaa_task_id = ankaa_task.id
ankaa_status = ankaa_task.state()

# print task status
print(f'Status of task: {ankaa_status}\n')

# print task id for future reference
print(f'Task id: {ankaa_task_id}')

In [ ]:
# get task arn
ankaa_task_arn = ''

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# print status
status = ankaa_task_id.state()
print("Status of the task:", status)

print(ankaa_task_id.state())
print(ankaa_task_id.metadata())  # Includes status, failure reason

<h1 style="color:#4E2A84;">Analyze Results</h1>



In [ ]:
# get task arn
ankaa_task_arn = ''

# load the quantum task
ankaa_task_id = AwsQuantumTask(arn=ankaa_task_arn)

# get results
ankaa_results = ankaa_task_id.result()

# get measurement counts
ankaa_counts = ankaa_results.measurement_counts
print("Measurement counts:", ankaa_counts)

In [ ]:
# import json to help pretty print dictionaries
import json

# helper function to check if nth bit is set in a string representing a binary number
# note bit counting starts at 0, i.e. 0th bit is the least significant bit
def is_nth_bit_set(binary_str, n):
    # Reverse the string to access LSB at index 0
    reversed_str = binary_str[::-1]

    if n >= len(reversed_str):
        return False  # Bit doesn't exist

    return reversed_str[n] == '1'

# helper function to check if bits at specific position match a pattern
def is_bit_pattern_set(binary_str, start, end, pattern_str):
    return binary_str[start:end] == pattern_str

counts_separated_ankaa = dict()

for i in range(num_copies):
    
    temp_counts = dict()
    
    for j in range(2**num_qubits):
        binary_str = bin(j)[2:].zfill(num_qubits)
        temp_counts[binary_str] = 0
    
    for j in range(2**num_qubits):
        for key in ankaa_counts.keys():
            binary_str = bin(j)[2:].zfill(num_qubits)
            if is_bit_pattern_set(key, i*num_qubits, (i+1)*num_qubits, binary_str):
            # print(i,j,binary_str)
                temp_counts[binary_str] = temp_counts[binary_str] + ankaa_counts[key]
    
    counts_separated_ankaa[i] = temp_counts

print(json.dumps(counts_separated_ankaa, indent=4))